In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
google_api_key=os.getenv("GOOGLE_API_KEY")


In [3]:
if google_api_key=='':
    print('api key not found')
else:
    print('api key found')

api key found


In [4]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.gemini import Gemini
from IPython.display import Markdown, display
from llama_index.core import ServiceContext
from llama_index.core import StorageContext, load_index_from_storage
import google.generativeai as genai
from llama_index.embeddings.gemini import GeminiEmbedding

In [5]:
genai.configure(api_key=google_api_key)

In [6]:
for models in genai.list_models():
    print(models)

Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko',
      description='Obtain a distributed representation of a text.',
      input_token_limit=1024,
      output_token_limit=1,
      supported_generation_methods=['embedText', 'countTextTokens'],
      temperature=None,
      max_temperature=None,
      top_p=None,
      top_k=None)
Model(name='models/gemini-1.0-pro-vision-latest',
      base_model_id='',
      version='001',
      display_name='Gemini 1.0 Pro Vision',
      description=('The original Gemini 1.0 Pro Vision model version which was optimized for '
                   'image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. '
                   'Move to a newer Gemini version.'),
      input_token_limit=12288,
      output_token_limit=4096,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=0.4,
      max_temperature=None,
      top_p=1.0,
     

In [7]:
for models in genai.list_models():
    if'generateContent' in models.supported_generation_methods:
        print(models.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-p

In [8]:
documents = SimpleDirectoryReader('../Data')
documents.load_data


<bound method SimpleDirectoryReader.load_data of <llama_index.core.readers.file.base.SimpleDirectoryReader object at 0x000001D818340C20>>

In [9]:
doc = documents.load_data()

In [10]:
print(doc[0].text)

GitHub.
User name - Vedd94
Password - 143mylife@123


Steam
user name - ii__ved__ii
Password - Asv@12345

HackerRAnk
password - Asv@12345

Valorent
username - WinterMaguire007
password - 143mylife@123

TPIN - 170556


In [11]:
model = Gemini(models='gemini-pro', api_key=google_api_key)
gemini_embed_model=GeminiEmbedding(model_name = 'models/embedding-001')

C:\Users\Ved Dahale\AppData\Local\Temp\ipykernel_18564\3909338345.py:1: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  model = Gemini(models='gemini-pro', api_key=google_api_key)
C:\Users\Ved Dahale\AppData\Local\Temp\ipykernel_18564\3909338345.py:2: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  gemini_embed_model=GeminiEmbedding(model_name = 'models/embedding-001')


In [12]:
service_context = ServiceContext.from_defaults(llm=model, embed_model = gemini_embed_model,chunk_size = 400, overlap = 20)

ValueError: ServiceContext is deprecated. Use llama_index.settings.Settings instead, or pass in modules to local functions/methods/interfaces.
See the docs for updated usage/migration: 
https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/

In [13]:
from llama_index.core import Settings

# Set these globally
Settings.llm = model
Settings.embed_model = gemini_embed_model
Settings.chunk_size = 400
Settings.chunk_overlap = 20

service_context = [Settings.llm, Settings.embed_model, Settings.chunk_size, Settings.chunk_overlap]


In [15]:
index = VectorStoreIndex.from_documents(doc, service_context = service_context)

In [16]:
index

In [17]:
index.storage_context.persist()

In [18]:
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query('What is my valorant account password?')

In [22]:
print(response.response)

The password for the Valorant account is 143mylife@123.



In [23]:
query_engine.query('Who is MS Dhoni')

Response(response='The provided text does not contain information about MS Dhoni.\n', source_nodes=[NodeWithScore(node=TextNode(id_='3bd78548-cd48-4c6d-90db-e38416ef7582', embedding=None, metadata={'file_path': 'd:\\GenAI\\QAsystem_llama_gemini\\notebook\\..\\Data\\Imp.txt', 'file_name': 'Imp.txt', 'file_type': 'text/plain', 'file_size': 231, 'creation_date': '2025-06-25', 'last_modified_date': '2022-02-14'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ea3f495c-1c1a-4ff0-8785-67b9ac8c9cef', node_type='4', metadata={'file_path': 'd:\\GenAI\\QAsystem_llama_gemini\\notebook\\..\\Data\\Imp.txt', 'file_name': 'Imp.txt', 'file_type': 'text/plain', 'file_size': 231, 'creation_date': '2025-06-25', 'last_modifie